In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the API key from the environment variable
api_key = os.getenv('GROQ_API_KEY')
if not api_key:
    raise ValueError("GROQ_API_KEY is not set in the .env file")

# Use the API key
os.environ['GROQ_API_KEY'] = api_key

# Initialize the ChatGroq object
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
)

response = llm.invoke("The first person to land on the moon was . . .")
print(response.content)

...Neil Armstrong! On July 20, 1969, Neil Armstrong became the first person to set foot on the moon, famously declaring "That's one small step for man, one giant leap for mankind" as he stepped off the lunar module Eagle onto the moon's surface.


In [2]:
import os
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"


In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://amazon.jobs/en/jobs/2926179/real-estate-development-manager-strategy-and-investment-data-center-supply-solutions")
page_data = loader.load().pop().page_content
print(page_data)

Real Estate Development Manager, Strategy and Investment, Data Center Supply Solutions - Job ID: 2926179 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsInclusive experiencesInterview tipsLeadership principlesWorking at AmazonFAQ×Real Estate Development Manager, Strategy and Investment, Data Center Supply SolutionsJob ID: 2926179 | ADSIPL - MaharashtraApply nowDESCRIPTIONAWS Infrastructure Services owns the design, planning, delivery, and operation of all AWS global infrastructure. In other words, we’re the people who keep the cloud running. We support all AWS data centers and all of the servers, storage, networking, power, and cooling equipment that ensure our customers have continual access to the innovation they rely on. We work on the most challenging problems, with thousands of variables impacting the supply chain — and we’re looking for talented people who

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format 
    containing the following keys: `role`, `experience`, `skills`, and `description`.
    Only return the valid JSON.
    
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data' :page_data})
print(res.content)

Here is the extracted job posting in JSON format:

```
{
  "role": "Real Estate Development Manager, Strategy and Investment, Data Center Supply Solutions",
  "experience": "10+ years of total work experience in the construction, datacenter, or real estate industries",
  "skills": [
    "Working knowledge of Indian developer and construction market",
    "Solid understanding of local permitting and development approvals",
    "Experience in reading and interpreting construction related drawings",
    "Experience managing schedules and costs in complicated multistory developments",
    "Project Management, Contract Administration, Quantity Surveying or other similar certification",
    "Exceptional process management skills",
    "Ability to work independently and prioritize effectively"
  ],
  "description": "The AWS Data Center Supply Solutions team is looking for a Real Estate Development Manager to drive our infrastructure strategy execution in India. As a Real Estate Development Ma

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format 
    containing the following keys: `role`, `experience`, `skills`, and `description`.
    Only return the valid JSON.
    
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data' :page_data})
type(res.content)

str

In [14]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Real Estate Development Manager, Strategy and Investment, Data Center Supply Solutions',
 'experience': '10+ years of total work experience in the construction, datacenter, or real estate industries',
 'skills': ['Working knowledge of Indian developer and construction market',
  'Solid understanding of local permitting and development approvals',
  'Experience in reading and interpreting construction related drawings',
  'Experience managing schedules and costs in complicated multistory developments',
  'Project Management, Contract Administration, Quantity Surveying or other similar certification',
  'Exceptional process management skills',
  'Ability to work independently and prioritize effectively'],
 'description': 'The AWS Data Center Supply Solutions team is looking for a Real Estate Development Manager to drive our infrastructure strategy execution in India. As a Real Estate Development Manager with Amazon Web Services, you will have the exciting opportunity to help bu

In [15]:
type(json_res)

dict

In [16]:
import pandas as pd

df = pd.read_csv("Cold_email_data.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [18]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [19]:
job = json_res
job['skills']

['Working knowledge of Indian developer and construction market',
 'Solid understanding of local permitting and development approvals',
 'Experience in reading and interpreting construction related drawings',
 'Experience managing schedules and costs in complicated multistory developments',
 'Project Management, Contract Administration, Quantity Surveying or other similar certification',
 'Exceptional process management skills',
 'Ability to work independently and prioritize effectively']

In [20]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}]]

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Real Estate Development Management Solutions for AWS Data Center Supply Solutions

Dear Hiring Manager,

I came across the job posting for a Real Estate Development Manager, Strategy and Investment, Data Center Supply Solutions at AWS, and I believe AtliQ can be a valuable partner in fulfilling your requirements. With our expertise in process optimization, cost reduction, and heightened overall efficiency, we can help drive your infrastructure strategy execution in India.

At AtliQ, we have a proven track record of delivering tailored solutions that cater to the unique needs of our clients. Our team of experts has extensive experience in managing complex projects, ensuring timely delivery, and maintaining exceptional process management skills. We understand the importance of working knowledge of the Indian developer and construction market, solid understanding of local permitting and development approvals, and experience in reading and interpreting construction-related 